In [12]:
from DataEncoding import DataEncoding
from torch import nn
from Operations import Operations
from Encoder import Encoder
from Decoder import Decoder

In [13]:
from datasets import load_dataset
ds = load_dataset("wmt/wmt14", "fr-en", split="train", num_proc=4)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
translation = ds["translation"][:3]
english = [sentence["en"] for sentence in translation]
french = [sentence["fr"] for sentence in translation]


In [15]:
from concurrent.futures import ThreadPoolExecutor

# Function to encode data
def encode_data(data, encoder):
    return encoder.input_data()

# Create thread pool for parallel processing
with ThreadPoolExecutor(max_workers=2) as executor:
    eng_future = executor.submit(encode_data, english, DataEncoding(english))
    fr_future = executor.submit(encode_data, french, DataEncoding(french))

    eng_embedded = eng_future.result()
    fr_embedded = fr_future.result()

In [17]:
class Transformer:
    def __init__(self, encoder_layers: int, decoder_layers: int):
        self.encoder_layers = encoder_layers
        self.decoder_layers = decoder_layers
        self.encoder_weights = [[0,0,0,0]]*self.encoder_layers
        self.decoder_mask_weights = [[0,0,0,0]]*self.decoder_layers
        self.decoder_weights = [[0,0,0,0]]*self.decoder_layers
        self.operation = Operations()
    
    def forward(self, eng_embedded, fr_embedded):
        encoder = Encoder()
        decoder = Decoder()
        
        for i in range(self.encoder_layers):
            Wqe, Wke, Wve, Woe = self.operation.generate_self_weights_QKV(eng_embedded)
            self.encoder_weights[i] = [Wqe, Wke, Wve, Woe]
                    
        for i in range(self.decoder_layers):
            Wqmd, Wkmd, Wvmd, Womd = self.operation.generate_self_weights_QKV(fr_embedded)
            self.decoder_mask_weights[i] = [Wqmd, Wkmd, Wvmd, Womd]
            
            Wqd, Wkd, Wvd, Wod = self.operation.generate_cross_weights_QKV(fr_embedded, eng_embedded)
            self.decoder_weights[i] = [Wqd, Wkd, Wvd, Wod]
        
        outpt_probs = None  
        for layer in range(self.encoder_layers):
                eng_embedded = encoder.forward(eng_embedded, self.encoder_weights[layer])
            
        for _ in range(self.decoder_layers):
            fr_embedded = decoder.forward(fr_embedded, eng_embedded, self.decoder_weights[layer], self.decoder_mask_weights[layer])
    
        fr_embedded = nn.Linear.forward(fr_embedded)
        outpt_probs = nn.Softmax.forward(fr_embedded)
        
        return outpt_probs
    
    def train(self, eng_embedded, fr_embedded):
        outpt_probs = self.forward(eng_embedded, fr_embedded)
        return outpt_probs
        

In [18]:
transformer = Transformer(encoder_layers=6, decoder_layers=6)
outpt_probs = transformer.train(eng_embedded, fr_embedded)
print(outpt_probs)



YES
tensor([], size=(0, 37, 512), grad_fn=<SliceBackward0>)


IndexError: index 0 is out of bounds for dimension 0 with size 0